In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

from sklearn.preprocessing import StandardScaler,MinMaxScaler 


In [3]:
TIME_STEPS=300

path = "train.csv"
text_file_reader = pd.read_csv(path, float_precision='round_trip', header=None, chunksize = TIME_STEPS)

In [7]:


def get_batch_data():
    columns = ['acoustic_data','time_to_failure']
    features = ['acoustic_data']

    #print("DEBUGGGGG")
    #print(path)
    df = text_file_reader.get_chunk(TIME_STEPS)
    df.columns = columns
    #df['rolling_100'] = df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    df['acoustic_data'] = df['acoustic_data'].astype(float)
    df[features] = StandardScaler().fit_transform(df[features])
    #print("debug", df['time_to_failure'].values[-1])
    return df[features].values.reshape(1,TIME_STEPS,1), np.array(df['time_to_failure'].values[-1]).reshape(1,1)


def get_generated_batch(sequence_idx):
    return get_batch_data()

In [8]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, split_filenames):
        self.split_filenames = split_filenames

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return 1258290

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
        train,Y = get_generated_batch(idx)
        #print("idx",idx)
        #print("LOLILOL")
        #print(train.shape, Y.shape)

        return (train,Y)

In [9]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2


my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,\
                  units = 3,\
                  stateful=True,batch_input_shape=(1,TIME_STEPS,1),\
                  #dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()

from keras.callbacks import Callback

class Reseter(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.reset_states()


callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0), Reseter()
]

#history = my_model.fit(train, Y, batch_size=BATCH_SIZE, epochs=100, shuffle=False,
                      #validation_data=(val_data,y_val), callbacks=callbacks
                    #  )
my_training_batch_generator = MY_Generator("split")
        
my_model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=1258290,
                                          epochs=10,
                                          shuffle=False,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          #use_multiprocessing=True,
                                          #workers=8,
                                          #max_queue_size=32
                      )

import matplotlib.pyplot as plt


#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (1, 3)                    60        
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    4         
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
    860/1258290 [..............................] - ETA: 27:12:48 - loss: 0.3413 - mean_absolute_error: 0.3413

KeyboardInterrupt: 

In [ ]:
next(my_training_batch_generator)